In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Импорт библиотек/фукнций из библиотек
# Уже импортированы numpy - алгебра и pandas - работа с csv файлами, табличками
import matplotlib.pyplot as plt #билиотека, строить графики
from sklearn.preprocessing import StandardScaler #Стандартизация характеристик
from sklearn.preprocessing import MinMaxScaler #Стандартизация характеристик
from sklearn.model_selection import KFold #Функция из библиотеки sklearn для разделения выборки для кросс-валидации
from sklearn.linear_model import SGDRegressor #Функция потерь для анализа одной выборки
from sklearn.metrics import mean_squared_error #MSE metrics для усреднения оценок кросс-валидации

In [ ]:
# Настройка изображения табличек
pd.options.display.max_columns = 200 #pd - Библиотека pandas для табличек
pd.options.display.max_rows = 100 # ПОЧЕМУ-ТО ТАК НЕ ПОЛУЧАЕТСЯ ВЫВЕСТИ 100 СТРОК :С

In [ ]:
# Чтение документов. Такой тип данных называется dataframe, читается с помощью библиотеки pandas
# Мы присваиваем переменной результат прочтения документа?
train=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv") 
test=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
sample_submission=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")

In [ ]:
train.head(10) # чтобы вывести 10 строк, почему-то так работает. Можно просто написать train и будет работать. Почему не нужно писать print()?

In [ ]:
test.head(10)

In [ ]:
sample_submission.head(10)

In [ ]:
#Cтандартизация данных (и тренировочных и тестовых) от -1 до 1 (среднее значение 0, дисперсия 1)
#mean = train.mean(axis=0)
#std = train.std(axis=0)
#Вычитаем среднее значение, делим на стандартное отклонение
#train -= mean
#train /= std
#Это не работает. Может из-за того, что в train есть столбцы с буквами и из-за этого нужно изменять по столбцу, а не всю таблицу.

In [ ]:
features=["GarageArea", "MasVnrArea", "GrLivArea", "YearRemodAdd", "GarageYrBlt", "YrSold", "MoSold", "SalePrice"]
#Обязательно ли создавать лист с csv файлами? Нельзя их сразу вытаскивать из таблицы?

In [ ]:
#Строим гистограммы для признаков, которые интересны, которые хотим использовать
plt.hist( train["GarageArea"], bins = 100)
plt.title ("GarageArea")
plt.show()
plt.hist( train["MasVnrArea"], bins = 100)
plt.title("MasVnrArea")
plt.show()
plt.hist( train["GrLivArea"], bins = 100)
plt.title ("GrLivArea")
plt.show()
plt.hist( train["YearRemodAdd"], bins = 100)
plt.title("YearRemodAdd")
plt.show()
plt.hist( train["GarageYrBlt"], bins = 100)
plt.title("GarageYrBlt")
plt.show()
plt.hist( train["MoSold"], bins = 100)
plt.title ("MoSold")
plt.show()
plt.hist( train["YrSold"], bins = 100)
plt.title("YrSold")
plt.show()
plt.hist( train["LotArea"], bins = 100)
plt.title("LotArea")
plt.show()
plt.hist( train["YearBuilt"], bins = 100)
plt.title("YearBuilt")
plt.show()
#Какое-то черное окно. Не понимаю, почему. Но вроде все остальное ок при этом.

In [ ]:
#np.corrcoef("GarageArea","Sale Price")
#Почему не работает???

In [ ]:
#Визуализируем корреляцию цены с интересующими характеристиками
plt.scatter( np.log1p(train ["GarageArea"]), np.log1p(train ['SalePrice']), s = 1 )
plt.title('Sale Price|GarageArea')
plt.show()
plt.scatter( np.log1p(train ["MasVnrArea"]), np.log1p(train ['SalePrice']), s = 1 )
plt.title('Sale Price|MasVnrArea')
plt.show()
plt.scatter( np.log1p(train ["GrLivArea"]), np.log1p(train ['SalePrice']), s = 1 )
plt.title('Sale Price|GrLivArea')
plt.show()
plt.scatter( np.log1p(train ["YearRemodAdd"]), np.log1p(train ['SalePrice']), s = 1 )
plt.title('Sale Price|YearRemodAdd')
plt.show()
plt.scatter( np.log1p(train ["GarageYrBlt"]), np.log1p(train ['SalePrice']), s = 1 )
plt.title('Sale Price|GarageYrBlt')
plt.show()
plt.scatter( np.log1p(train ["MoSold"]), np.log1p(train ['SalePrice']), s = 1 )
plt.title('Sale Price|MoSold')
plt.show()
plt.scatter( np.log1p(train ["YrSold"]), np.log1p(train ['SalePrice']), s = 1 )
plt.title('Sale Price|YrSold')
plt.show()
plt.scatter( np.log1p(train ["LotArea"]), np.log1p(train ['SalePrice']), s = 1 )
plt.title('LotArea|YrSold')
plt.show()
plt.scatter( np.log1p(train ["YearBuilt"]), np.log1p(train ['SalePrice']), s = 1 )
plt.title('YearBuilt|YrSold')
plt.show()

In [ ]:
#Выбираем характеристики, коррелирующие со стоимостью домов. Есть признаки, хорошо коррелирующие, но не на всем промежутке. Что с ними можно сделать? Можно ли отбросить части значений, где они плохо предсказывают y? (если признак "площадь гаража" равен 0, например)
#Для хорошей линейной регрессии нужно нормальное распределение остатков, гомоскедатичность. Если мы предсказываем по нескольким независимым параметрам, то они должны плохо коррелировать между собой.
#Выбираю GarageArea, MasVnrArea, GrLivArea, YearRemodAdd, GarageYrBlt

In [ ]:
train["SalePriceLog"] = np.log1p(train["SalePrice"])
#Стандартизуем данные как на занятии показали и в тренировочном и в тестовых наборах
#Цель - уменьшить диапазон чисел и привести распределение +- к нормальному
#Можно использовать Z-стандартизацию или зажимать значения в отрезке или логнормальное распределение использовать
#Как понять, что использовать?
#Я решила по гистограммам посмотреть и с сильно ассиметричными использовать StandardScaler, а с более менее норм MinMax

Lg_scaler =  StandardScaler() #что делаем этой строчкой?
test['GarageAreaLog'] = np.log(test['GarageArea'] + 1) #Сначала Логарифмируем, чтобы уменьшить значения. Но зачем прибавляем единицу???
train['GarageAreaLog'] = np.log(train['GarageArea'] + 1)#Создаем новый столбец в таблице, значения в столбце будут равняться np.log от столбца (LogArea +1)
train['GarageAreaNormalized'] = Lg_scaler.fit_transform(train[['GarageAreaLog']])#Почему в этой строчке fit_transform
test['GarageAreaNormalized'] = Lg_scaler.transform(test[['GarageAreaLog']])#а в этой просто transform?
#Почему при стандартизации нельзя сделать так же как и  при логарифмировании? Почему в три странные строчки это действие выполняется? 

test['MasVnrAreaLog'] = np.log(test['MasVnrArea'] + 1)
train['MasVnrAreaLog'] = np.log(train['MasVnrArea'] + 1)
train['MasVnrAreaNormalized'] = Lg_scaler.fit_transform(train[['GarageAreaLog']])
test['MasVnrAreaNormalized'] = Lg_scaler.transform(test[['GarageAreaLog']])

#MinMaxScaler`ом зажимаем все значения в интервале от 0 до 1 (точек в таком интервале больше, чем рациональных чисел - поэтому мы так можем сделать?)
MMScal =  MinMaxScaler() 

train['GrLivAreaNormalized'] = MMScal.fit_transform(train[['GrLivArea']])
test['GrLivAreaNormalized'] = MMScal.transform(test[['GrLivArea']])

train['YearRemodAddNormalized'] = MMScal.fit_transform(train[['YearRemodAdd']])
test['YearRemodAddNormalized'] = MMScal.transform(test[['YearRemodAdd']])

train['GarageYrBltNormalized'] = MMScal.fit_transform(train[['GarageYrBlt']])
test['GarageYrBltNormalized'] = MMScal.transform(test[['GarageYrBlt']])
#После этого можно еще раз визуализировать, чтобы понимать, что изменилось

In [ ]:
model = SGDRegressor() #заключаем функцию в переменную? что это? Такая же строчка как при стандартизации

In [ ]:
#Это примерный смысл того, что мы будем делать дальше. Но в начале мы проводим кросс-валидацию
#model.fit (train [features], train [["SalePriceLog"]] ) #здесь мы и используем весь наш лист features похоже. Видимо мы создавали его как раз для этого. Находим коээффициенты графика, где y - цена, которую мы предсказываем (второй параметр), а х - независимая переменная (первый параметр). Делаем это все на тренировочных данных.
#model.predict (train [features]) #Что делает эта команда?
#train_prediction = model.predict( X = train[['YearBuiltNormalized', 'LogLotAreaNormalized'] ] ) #Тут мы вводим обозначение функции, которая не знаю что делает
#model.intercept_ #коэффициент b0
#model.coef_ #коэффициент b1

In [ ]:
kf = KFold(n_splits=5) #обозначаем функцию, делящую данные на 5 частей
kf.get_n_splits(train) #делим данные train на 5 частей

mse_list = [] #обозначили пустой лист
rmse_list = []
for i, (train_index, test_index) in enumerate(kf.split(train)): #не очень поняла строчку
    print( "Fold: ", i)
    #далее мы похоже просто вводим переменные x_train y_train и в них находятся независимые переменные и предсказываемая, соответственно, из столбцов таблицы
    #Что значит train.loc?
    x_train, x_test = train.loc[train_index, ['GarageAreaNormalized']], train.loc[test_index, ['GarageAreaNormalized']]
    y_train, y_test = train.loc[train_index, 'SalePriceLog'],train.loc[test_index, 'SalePriceLog']
                                 
    #Вот на этом месте я могу менять характеристики по которым предсказываю интересующий меня показатель и комбинировать их.
    
    model.fit (x_train, y_train)
    x_test_predictions = model.predict (x_test)
    mse = mean_squared_error (y_true = y_test, y_pred = x_test_predictions)#Обозначаем параметры метрики y_true и y_pred
    print(i, mse)
    mse_list.append(mse)#для каждого кусочка данных рассчитывается mse и добавляется в лист
    rmse_list.append(mse)
    
    plt.scatter(x_test_predictions, y_test, s = 2)
    plt.ylabel('true value')
    plt.xlabel('predicted value')
    plt.plot(y_test, y_test, c='r')
    plt.show()

In [ ]:
mse_list #выводит значения ошибок, которые на предыдущем шаге туда собрались

In [ ]:
print (np.mean(mse_list), np.std(mse_list)) #Зачем стандартное отклонение?
print (np.mean(rmse_list), np.std(rmse_list))

In [ ]:
model.fit(train.loc[:, [ 'GarageAreaNormalized']], train.loc[:,  'SalePriceLog']) #зачем двоеточия?

In [ ]:
test_predictions = model.predict(test.loc[:, ['GarageAreaNormalized']]) #предсказываем SalePrice по GarageAreaLog из тестовой выборки

In [ ]:
sample_submission['SalePrice'] = np.exp(test_predictions)-1 #создаем столбец в dataframe sample_submission
sample_submission.to_csv('submission2.csv', index = None) #сохраняем как csv файл

In [ ]:
sample_submission #Выводит файл, в который мы должны зписать полученные результаты на тестовой выборке